This notebook transform Q&A pairs into co-occurrency tables, counting the frequency of each non-stopword term in question and answer

In [1]:
import psycopg2
conn = psycopg2.connect(user="postgres", password="mysecretpassword", host="localhost")
# distinct connection to perform insertions while iterating over the SELECT results
insertion_conn = psycopg2.connect(user="postgres", password="mysecretpassword", host="localhost")
# should be UTF-8
print('connection encoding, used as default:', conn.encoding)

connection encoding, used as default: UTF8


In [2]:
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS words_tmp (word_q VARCHAR(30), word_a VARCHAR(30), counter INTEGER);")
cur.execute("CREATE TABLE IF NOT EXISTS bigrams_tmp (token1 VARCHAR(30), token2 VARCHAR(30), counter INTEGER);")
cur.execute("TRUNCATE TABLE words_tmp;")
cur.execute("TRUNCATE TABLE bigrams_tmp;")
conn.commit()

Use a named cursor `window_read_cursor` to read the conversation by channel sort by date, so that consecutive messages in the same channel come together.

The named cursor is necessary to iterate over an important amount of data, which would not easily fit into memory

In [3]:
import time
import random
from datetime import datetime

named_cur = conn.cursor('window_read_cursor')
insertion_cur = insertion_conn.cursor()


# table schema:
# chat_messages (message TEXT, message_time TIMESTAMP, username TEXT, channel TEXT)
print("about to run query")
named_cur.execute("SELECT question, answer FROM qa_pair")


start_time = time.time()

print("iterating over query results")
total = 0
to_store = {}
to_store_bigrams = {}

stopwords_it = set(["a","ad","agl","agli","ai","al","alcuna","alcuni","alcuno","all","alla","alle","allo","allora","altri","altro","anche","ancora","aver","avere","averlo","avesse","avessero","avessi","avessimo","aveste","avesti","avete","aveva","avevamo","avevano","avevate","avevi","avevo","avrai","avranno","avrebbe","avrebbero","avrei","avremmo","avremo","avreste","avresti","avrete","avrà","avrò","avuta","avute","avuti","avuto",
                    "basta","è","il","le", "che", "un","una","uno","perché","mi","di","ma","per","in","non","e","si","sto","la","lo","gli","con","nel","già","da","i","sono","come","se","io","ho","più","però","ti","su","hai","fa","per","tra","fra","me","ha","ci","del","poi","o","cosa","cose","mai","fare"])

# do not filter out stopwords for now, keep the raw data
stopwords_it = set()

external_token = '**EXTERNAL**'

for record in named_cur:
    tokens_q = set(record[0].lower().split())
    tokens_a = set(record[1].lower().split())
    for q in tokens_q:
        if q in stopwords_it or len(q) > 30:
            continue
        for a in tokens_a:
            if a in stopwords_it or len(a) > 30:
                continue
            if (q,a) not in to_store:
                to_store[(q,a)] = 0 
            to_store[(q,a)] += 1

            
    padded = [external_token] + record[0].lower().split() + [external_token]
    for pos in range(len(padded) -1 ):
        bg = (padded[pos][:30], padded[pos + 1][:30])
        if bg not in to_store_bigrams:
            to_store_bigrams[bg] = 0
        to_store_bigrams[bg] += 1
        
    padded = [external_token] + record[1].lower().split() + [external_token]
    for pos in range(len(padded) -1 ):
        bg = (padded[pos][:30], padded[pos + 1][:30])
        if bg not in to_store_bigrams:
            to_store_bigrams[bg] = 0
        to_store_bigrams[bg] += 1
    total += 1
    if len(to_store) + len(to_store_bigrams) > 150000:
        print(f'{datetime.now().isoformat()} - insertion after {total} pairs')
        for pair,count in to_store.items():
            insertion_cur.execute("INSERT INTO words_tmp (word_q, word_a, counter) VALUES (%s, %s, %s)", (pair[0], pair[1], count))
        for pair,count in to_store_bigrams.items():
            insertion_cur.execute("INSERT INTO bigrams_tmp (token1, token2, counter) VALUES (%s, %s, %s)", (pair[0], pair[1], count))
   
        to_store = {}
        to_store_bigrams = {}

        insertion_conn.commit()
    
insertion_conn.commit()
insertion_cur.close()


elapsed_time = round(time.time() - start_time)
print(f'{total} pairs examined in {elapsed_time} seconds')

about to run query
iterating over query results
insertion after 1827 pairs
insertion after 3388 pairs
insertion after 4967 pairs
insertion after 6747 pairs
insertion after 8306 pairs
insertion after 11464 pairs
insertion after 13218 pairs
insertion after 14521 pairs
insertion after 15976 pairs
insertion after 17631 pairs
insertion after 19472 pairs
insertion after 21075 pairs
insertion after 22962 pairs
insertion after 24461 pairs
insertion after 26041 pairs
insertion after 27709 pairs
insertion after 29192 pairs
insertion after 30656 pairs
insertion after 32372 pairs
insertion after 34279 pairs
insertion after 35897 pairs
insertion after 37093 pairs
insertion after 38601 pairs
insertion after 39943 pairs
insertion after 41397 pairs
insertion after 42747 pairs
insertion after 44421 pairs
insertion after 45926 pairs
insertion after 47655 pairs
insertion after 50387 pairs
insertion after 52038 pairs
insertion after 53566 pairs
insertion after 55244 pairs
insertion after 56873 pairs
inser

In [4]:
start_time = time.time()
cur = conn.cursor()
cur.execute("CREATE TABLE words AS SELECT word_q, word_a, SUM(counter) as counter FROM words_tmp GROUP BY word_q, word_a;")
conn.commit()
cur.execute("CREATE TABLE bigrams AS SELECT token1, token2, SUM(counter) as counter FROM bigrams_tmp GROUP BY token1, token2;")
conn.commit()
cur.execute("DROP TABLE bigrams_tmp;")
cur.execute("DROP TABLE words_tmp;")
conn.commit()

elapsed_time = round(time.time() - start_time)
print(f'tables created, tmp tables dropped in {elapsed_time} seconds')

start_time = time.time()

cur.execute("CREATE INDEX question_w ON words(word_q);")
conn.commit()
cur.execute("CREATE INDEX answer_w ON words(word_a);")
conn.commit()

cur.execute("create table word_counts as select sum(counter) as total, token1 as token from bigrams group by token1;")
conn.commit()

cur.execute("""create table relevances as (select word_a, word_q, counter/(wc_a.total + wc_q.total) as relevance
FROM words
JOIN word_counts wc_a ON wc_a.token = word_a
JOIN word_counts wc_q ON wc_q.token = word_q);""")
conn.commit()


elapsed_time = round(time.time() - start_time)
print(f'aggregate table created in {elapsed_time} seconds')

tables created, tmp tables dropped in 1219 seconds
aggregate table created in 272 seconds
